<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<p>
<h1>Algorísmica Avançada</h1>
<h2>Parcial Práctico 2 - Enumerativos </h2>
</center>
</p>
</div>

<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h2><p>Saltos de caballo</p></h2>
  
  <p>
 En el ajedrez, un caballo se mueve en forma de "L" en cualquier dirección posible. Es decir, se mueve en línea recta 2 posiciones y luego 1 más a izquierda o derecha. Todos los movimientos posibles de un caballo en una posición dada puden verse en la siguiente imagen:

<p><center><img src='img/horse_moves.jpg' width=30%></img></center></p>

<p>El examen consta de un único ejercicio en el cual tendréis que implementar un algoritmo de ramificación y poda (recursivo o iterativo) que, dado un tablero de tamaño $n \times n$ y un número de casillas $C$, devuelva el recorrido de caballo más corto que pasa por $C$ casillas diferentes (contando la inicial, la cual es el tercer parámetro de la función).
</p>

Os damos ya implementada la clase HorseState. Podéis modificar la clase como considréis necesario o podéis incluir otras funciones de interés para la resolución del problema. Si empleáis alguna variable global recordad que es necesario declararlas dentro de las funciones como <i>global</i> en caso de que queramos modificarlas. Por ejemplo:

```
global_variable = 0

def function_that_reads_global_var(param):
    if global_variable > param:
        print("It is greater!")

def function_that_modifies_global_var(param):
    global global_variable # OJO!
    if global_variable > param:
        global_variable = param
```

Por último, también os pedimos que retornéis la cantidad de estados que ha sido necesario explorar hasta encontrar la solución óptima.


In [7]:
# La función deepcopy(x) retorna una copia del objeto x 
# de manera que lo podemos modificar sin que afecte al original
from copy import deepcopy

# Os damos la clase HorseState como base para que realicéis vuestro trabajo
class HorseState:
    def __init__(self,N,S,C,sol=None,value=0):
        self.size = N # Tamaño del tablero
        self.objective = C # Número de celdas distintas a visitar
        self.sol = sol if sol else [S]  # Camino construido hasta ahora, la posición actual del caballo es self.sol[-1]
        self.value = value + 1 # Peso total de la solución (equivalente a len(self.sol))
    
    # Devuelve las posiciones accesibles para un caballo desde la celda actual (la última añadida a la solución).
    def possible_next(self):
        (x,y) = self.sol[-1]
        possible = []
        if x < self.size and y < self.size:
            if x-2 >= 0:
                if y-1 >= 0:
                    possible.append((x-2,y-1))
                if y+1 < self.size:
                    possible.append((x-2,y+1))
            if x+2 < self.size:
                if y-1 >= 0:
                    possible.append((x+2,y-1))
                if y+1 < self.size:
                    possible.append((x+2,y+1))
            if y-2 >= 0:
                if x-1 >= 0:
                    possible.append((x-1,y-2))
                if x+1 < self.size:
                    possible.append((x+1,y-2))
            if y+2 < self.size:
                if x-1 >= 0:
                    possible.append((x-1,y+2))
                if x+1 < self.size:
                    possible.append((x+1,y+2))
        return possible
    
    # Devuelve un nuevo estado añadiendo la celda
    def add_cell(self,cell):
        #if not cell in self.sol:
        return HorseState(self.size,self.sol[0],self.objective,self.sol+[cell],self.value)
        #else:
        #    return HorseState(self.size,self.sol[0],self.objective,self.sol,self.value)
    
    # LessThan compara el objeto actual con otro 
    # (lo utilizan clases como PriorityQueue o HeapQ para ordenar los objetos)
    def __lt__(self,other):
        return self.value < other.value
    
    # Convierte el objeto a string (para formatear los prints)
    def __str__(self):
        return str(self.sol) + " " + str(self.value)
            

In [14]:
'''
Implementad aquí vuestra solución:
@input:     N --> tamaño del tablero.
            S --> posición inicial del caballo en forma de tupla.
            C --> número de celdas diferentes a recorrer (como máximo N²).
@output:    path --> Lista de celdas que representan el recorrido mínimo pasando por C celdas diferentes.
            expanded --> Número de nodos explorados antes de encontrar la solución óptima.
''' 
def min_horse_path(N,S,C):
    H = HorseState(N,S,C)
    
    res,cota_sup,nodes = [],N**2+1,0
    res.insert(0,H)
    
    while(res):
        
        current = res.pop(0)
        nodes+=1
        
        if len(current.sol)==current.objective:
            if current.value<cota_sup:
                return current.sol,nodes
                cota_sup = current.value
                solucio = current.sol,nodes
                # print(solucio,cota_sup)
    
        for possibleMov in current.possible_next():
            if current.value+1<cota_sup and not possibleMov in current.sol:
                res.insert(0,current.add_cell(possibleMov))
    
    return solucio

In [15]:
import time
start_time = time.time()
path,n = min_horse_path(5,(0,0),25)
t = time.time() - start_time
assert len(path) == 25
print(path,n)
print("--- %s seconds ---" % (t))

[(0, 0), (1, 2), (2, 4), (4, 3), (3, 1), (1, 0), (2, 2), (0, 3), (1, 1), (3, 0), (4, 2), (3, 4), (1, 3), (0, 1), (2, 0), (4, 1), (3, 3), (1, 4), (0, 2), (2, 1), (4, 0), (3, 2), (4, 4), (2, 3), (0, 4)] 74301
--- 0.5582535266876221 seconds ---


<div class="alert alert-warning" style="width:80%; margin:0 auto; padding">
<center><p><h3> Comentarios y análisis de complejidad</h3></p> </center> </div>

<b> Explica brevemente la estrategia que has empleado para resolver el problema, enfatizando los mecanismos que reducen el espacio de búsqueda, y realiza el análisis de complejidad de tu algoritmo. </b>

<div class="alert alert-info">
<center>
  <h1>Entrega</h1>
</center>
<p>
Tenéis 1h30m para resolver el ejercicio. Se habilitará una tarea en el Campus Virtual pasada media hora desde el comienzo del examen. Una vez terminéis, subid el examen a la tarea "Segundo parcial de pácticas" con vuestro nombre y el grupo de pácticas al que asistís:
</p>
<p>
```
AA_EXP2_<grupo_practicas>_<apellido1>_<apellido2>_<nombre>.ipynb
```

</p>
<p>
    Por ejemplo, para una tal <b>Marina Perez García</b> perteneciente al grupo <b>B</b> de prácticas, su archivo deberá llamarse:
</p>
<p>
```
AA_EXP2_B_Perez_Garcia_Marina.ipynb
```

</p><p>
Nota:<b> NO ADHERIRSE AL FORMATO DEL NOMBRE PUEDE HACER QUE NOS OLVIDEMOS DE CORREGIR VUESTRO EXAMEN.</b></p>
</div>